In [ ]:
%tensorflow_version 1.x

import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/argman/EAST.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q $git_repo_url
  #!cd $project_name && pip install -q -r requirements.txt
  # patch for the Python 3.7
  !sed -i 's/python3-config/python3.7-config/g' EAST/lanms/Makefile
  
import sys
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

TensorFlow 1.x selected.


In [ ]:
def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name
  

pretrained_model = 'east_icdar2015_resnet_v1_50_rbox'
if not exists(pretrained_model):
  # download the pretrained model
  pretrained_model_file_name = 'east_icdar2015_resnet_v1_50_rbox.zip'
  download_from_google_drive('0B3APw5BZJ67ETHNPaU9xUkVoV0U', pretrained_model_file_name)
  !unzip -qq $pretrained_model_file_name
  !mkdir /tmp/east_icdar2015_resnet_v1_50_rbox
  !mv east_icdar2015_resnet_v1_50_rbox/* /tmp/east_icdar2015_resnet_v1_50_rbox/


!unzip -qq training_data.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   403    0   403    0     0    790      0 --:--:-- --:--:-- --:--:--   790
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  346M  100  346M    0     0   152M      0  0:00:02  0:00:02 --:--:--  346M


In [ ]:
!python EAST/multigpu_train.py --gpu_list=0 --input_size=512 --batch_size_per_gpu=14 \
--checkpoint_path=/tmp/east_icdar2015_resnet_v1_50_rbox/ \
--text_scale=512 --training_data_path=training_data --geometry=RBOX --learning_rate=0.0001 \
--num_readers=24 --restore --save_checkpoint_steps=100 --batch_size_per_gpu=10 --max_steps=300

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0105 17:38:38.488350 139812654860160 module_wrapper.py:139] From EAST/multigpu_train.py:72: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.


W0105 17:38:38.488598 139812654860160 module_wrapper.py:139] From EAST/multigpu_train.py:79: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0105 17:38:38.496618 139812654860160 module_wrapper.py:139] From EAST/multigpu_train.py:87: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.


W0105 17:38:38.500110 139812654860160 module_wrapper.py:139] From EAST/multigpu_train.py:88: The nam

In [ ]:
!ls /tmp/east_icdar2015_resnet_v1_50_rbox

checkpoint
events.out.tfevents.1641404328.de4d7887bfe4
model.ckpt-49491.data-00000-of-00001
model.ckpt-49491.index
model.ckpt-49491.meta
model.ckpt-49492.data-00000-of-00001
model.ckpt-49492.index
model.ckpt-49492.meta
model.ckpt-49593.data-00000-of-00001
model.ckpt-49593.index
model.ckpt-49593.meta
model.ckpt-49694.data-00000-of-00001
model.ckpt-49694.index
model.ckpt-49694.meta


In [ ]:
!zip checkpoint.zip /tmp/east_icdar2015_resnet_v1_50_rbox/model.ckpt-49694.data-00000-of-00001 /tmp/east_icdar2015_resnet_v1_50_rbox/model.ckpt-49694.index /tmp/east_icdar2015_resnet_v1_50_rbox/model.ckpt-49694.meta /tmp/east_icdar2015_resnet_v1_50_rbox/checkpoint

  adding: tmp/east_icdar2015_resnet_v1_50_rbox/model.ckpt-49694.data-00000-of-00001 (deflated 6%)
  adding: tmp/east_icdar2015_resnet_v1_50_rbox/model.ckpt-49694.index (deflated 72%)
  adding: tmp/east_icdar2015_resnet_v1_50_rbox/model.ckpt-49694.meta (deflated 94%)
  adding: tmp/east_icdar2015_resnet_v1_50_rbox/checkpoint (deflated 70%)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!cp /content/checkpoint.zip "/content/gdrive/MyDrive/training_data/meeter-reader/east"

Mounted at /content/gdrive
